<a href="https://colab.research.google.com/github/NbtKmy/ki_in_der_bibliothek/blob/main/ChatGPT_Anwendungsbeispiel_ZB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatGPT mit Embedding

KI lügt - das Phänomen "Halluzination" ist mittlerweile bekannt. Auch wenn LLMs immer besser werden, bleibt sie weiterhin noch ein grosses Probelem.
Wenn Bibliotheken einen Dienstleistung mit ein LLM wie ChatGPT konzipieren, ist eine Massnahme gegen die Halluzination notwendig.

Hier in diesem Beispiel versuchen wir die Halluzination zu reduzieren, indem wir Embedding verwenden.
(Die andere denkbare Massnahme ist Fine Tuning. Aber hier wird sie nicht berücksichtigt.)

## Embedding?

>Bei einer Worteinbettung (englisch word embedding) handelt es sich um eine Einbettung, bei der Worte oder andere Symbole jeweils einem Vektor v mit $v \in \mathbb{R}^n$ zugeordnet werden.

(Wikipeida [Worteinbettung](https://de.wikipedia.org/wiki/Worteinbettung))

... Worteinbettung (Vektorisierung der Wörter) ermöglicht die Berechnung der Bedeutungen der Wörter in einem Kopus...

__Illustration für Worteinbettung__

<img src="https://upload.wikimedia.org/wikipedia/commons/3/3f/Word_vector_illustration.jpg" width="400" />

(Word vector illustration.jpg by Singerep, aus [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Word_vector_illustration.jpg) CC-SA)



Die Embedding kann nicht nur Wörtern sondern auch Sätze/Text-Einheiten unterzogen werden.



Was wir hier in diesem Beispiel machen, sieht so aus:



<img src="https://nbtkmy.github.io/presentations/coffee_lectures/chatgpt_embedding.png" alt="embedding" width="400"/>


## Beispiel


Als Beispiel nehmen wir das PDF-Dokument vom 1. Bericht der ZB.

__Titel__: 1. Bericht der Zentralbibliothek Zürich (Öffentliche Stiftung) über die Jahre 1914-1917

__Urheber__: Zentralbibliothek Zürich


__Link__: https://doi.org/10.20384/zop-1421


![zb](https://zop.zb.uzh.ch/retrieve/e7ca61c6-c25b-407e-b0f0-d401efa961f5/990019340740205508-0001.pdf.jpg)





In [ ]:
# Libraries installieren - Wir verwenden hier Langchain
!pip install -q openai chromadb langchain pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done

In [ ]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader


In [ ]:
# PDF-Dokument nach der Seite trennen (man kann auch andere Einheit auswählen)
loader = PyPDFLoader("./zb_1914_1917.pdf")
pages = loader.load_and_split()
# Seite 5 anzeigen
pages[8].page_content

'erhob sich die für das Personal überaus wichtige Frage, ob\ndie bisherigen, imDienste der beidseitigen Bibliotheken ver—\nbrachten Jahre anzurechnen seien oder nicht. Wenn ja, s0\nhatte die Zentralbibliothek éin nicht unerhebliches Lintritts-\ndefizit zuübernehmen. Einrechtlicher Anspruch aufAnrechnung\ndeèrDienstjahre bestand selbstverständlich nicht; dagegen glaubten\ndieAngehörigen derStadtbibliothek wenigstens einen moralischen\neérheben zudürfen. Um derBibliothekbehörde dieZustimmung\nzuerleichtern, erklärte sieh dasPersonal fürden Fall derAn-\nrechnung der Dienstjabre beéreit, statt des den städtischen\nBeéamten auferlegten Jahresbeitrages von 1420,0 des Geéhalts,\néinen solchen von 29200 zuleisten und dasRentenbezugsalter\nvom 65. auf das vollendete 70.Altersjahr zuverschieben, was\ndie Prämieneinzahlungen der Zentralbibliotheß von 79,0 auf\n50herabzusetzen gestattete. Zudiesem Betrage kam freilich für\nden Arbeitsgeber noch dieVerzinsung des Eintrittsdefizits. Die\nBibliothekk

In [ ]:
pages[6].page_content

'Emporwachsen desNeubaus unwobnlich geworden waren. Dort\nwurden biszum Umzug dielaufenden Geschäfte érledigt, dort\ntraten dieBibliothekare zuden inderBibliothekordnung vor-\ngesehenen regelmäßigen Konferenzen zusammen, und dort war,\ntrotz derDeberfüllung derRäume, auch Plata zuschaffen für\nden neuen Zuvwachs.\nPersonal. Nach den getroffenen Vereinbarungen übernahm\ndieZentralbibliothek aufden 1.Januar 1916 dasPersonal der\nKantons- und der Stadtbibliothek. Für den neuen Beamten-\nkörper wurden einhbeitliche Anstellungs- und Dienstverhältnisse\naufgestellt. Neu gewählt wurde alsBibliothekar Herr Dr.Bruno\nHirzel von Züriech; alsBureaugebilfin, später alsBibliothek-\ngehiltfin Fräulein Clara Michel von Zürich.\nAuberhalb des festangestellten Personals istseit Sommer\n1917, Dank derHilfe eines Gönners, Herr Arthur Bueb von\nOltingen RKt.Baselland inderBibliothek beschäftigt. AlsVolon-\ntäre waren tätig: vom 1.April bis30.September 1916 Herr\nDr. Otto Gréuter vonWinterthur; vom 1.April

In [ ]:
os.environ["OPENAI_API_KEY"] = "[YOUR API KEY]"
openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [ ]:
# Wenn kein Modell hier genannt, dann wird "text-embedding-ada-002" genommen.
# Chroma wird hier als Vector Store verwendet
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(pages, embedding=embeddings, persist_directory=".")
vectorstore.persist()

In [ ]:
pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)


In [ ]:
query = "Für wen war die Revision der Titelaufnahmen wichtig?"
chat_history = []

result = pdf_qa({"question": query, "chat_history": chat_history})

result["answer"]

'Die Revision der Titelaufnahmen war für die drei kleineren Bibliotheken wichtig.'

In [ ]:
result["source_documents"]

[Document(page_content='— — —\nwährend Jahrzehnten der Stadtbibliothek lebhafte Teilnahme\nzugewandt hatte, gelangte einvon seiner imJahre 1894 ver-\nstorbenen Schwiegermutter Frau Plis. Hagenbuch-Ott hinter-\nlassenes Testament zurAusführung, das für den z.Zz.noch\nbesonders zuverwaltenden Stadtbibliothekfonds dieZuwendung\nvon PFr. 28,927. 20ergab und der Zentralbibliothek überdies\neine reiche, vom Vater derErblasserin angelegte MAutographen-\nsammlung 2zuführte.\nZum Andenken anVerstorbene kamen uns zu: von den\nHinterlasscnen des Herrn O.Abegg-Arter Fr.25,000. —; von\ndenen des Herrn H.O.Bodmer, éinstigen Münzdirektors der\nStadtbibliothek, die Summe von Fr.25,000. —; von den Ge-\nschwistern des Herru Dr. G.Finsler, Rektors des bernischen\nLiterargymnasiums, eine grobe AuswWahl vonWérken ausdessen\ninsbesoudere anHomeérliteratur reicher Bibliothek; von den\nHinterlassenen desHerru Emil Gebner inWädenswil Fr.5000. —.\nEinige fürdieBibliothek bedeutsame Zeitpunkte drückten\nsieh auc